In [1]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2021, day=19)

def parses(text):
    scanners = text.strip().split('\n\n')
    scanners = [[[int(i) for i in line.split(',')] for line in scanner.split('\n')[1:]] for scanner in scanners ]
    return [np.array(s) for s in scanners]

data = parses(puzzle.input_data)

In [36]:
sample = parses("""--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14""")

In [40]:
def righthand_rotations():
    # Generate all 3! * 2**3 possible rotation/mirror matrices
    # and keep the ones that are rotation since det(R) = 1
    I = np.eye(3, dtype=int)
    row_perms = map(np.array, itertools.permutations(range(3)))
    signs = map(np.array, itertools.product((1,-1),repeat=3))
    Rs = [I[p]*s for p, s in itertools.product(row_perms, signs)]
    return [R for R in Rs if np.linalg.det(R) == 1]

ROTATIONS = righthand_rotations()

In [42]:
def valid_overlap(known, candidate):
    known_set = set(map(tuple,known))
    # known is absolute coordinates, candidate is relative
    for rotation in ROTATIONS:
        for point in known:
            for distance in candidate:
                potential_scanner = point -  distance @ rotation
                potential_absolute = candidate @ rotation + potential_scanner
                potential_set = set(map(tuple, potential_absolute))
                if len(known_set & potential_set) >= 12:
                    return potential_scanner, potential_absolute

In [45]:
def find_absolute_positions(scanners):

    locations = {0: np.array([0,0,0])}
    absolute = {0: scanners[0]}
    unsolved = set(range(1,len(scanners)))
    tested = set()

    while len(unsolved) > 0:
        for known, candidate in itertools.product(absolute, unsolved):
            if (known, candidate) not in tested:
                tested.add((known, candidate))
                if (sol := valid_overlap(absolute[known], scanners[candidate])) is not None:
                    locations[candidate], absolute[candidate] = sol
                    unsolved.remove(candidate)
                    print(f'Found {candidate} using {known} at locations {locations[candidate]}')
                    break
    return locations, absolute 

In [46]:
import time
start = time.time()
find_absolute_positions(sample)
print(time.time()-start)

Found 1 using 0 at locations [   68 -1246   -43]
Found 3 using 1 at locations [  -92 -2380   -20]
Found 4 using 1 at locations [  -20 -1133  1061]
Found 2 using 4 at locations [ 1105 -1205  1229]
3.015087127685547


In [47]:
def solve_a(scanners):
    _, absolute = find_absolute_positions(scanners)
    return len(np.unique(np.vstack(list(absolute.values())), axis=0))

In [49]:
x = find_absolute_positions(data)

Found 2 using 0 at locations [ -28 1245 -161]
Found 24 using 0 at locations [1163   26 -170]
Found 27 using 0 at locations [-1254    90  -158]
Found 18 using 2 at locations [  -18  1355 -1280]
Found 31 using 24 at locations [ 1146 -1193   -28]
Found 1 using 27 at locations [-2380    15   -13]
Found 4 using 27 at locations [-1078    63  1010]
Found 26 using 27 at locations [-1105    28 -1236]
Found 7 using 18 at locations [  -12  2400 -1318]
Found 23 using 18 at locations [-1075  1190 -1385]
Found 15 using 1 at locations [-2348   173 -1313]
Found 32 using 1 at locations [-2450 -1024   -26]
Found 36 using 1 at locations [-3547     0   -79]
Found 11 using 4 at locations [-1233 -1211  1144]
Found 12 using 4 at locations [-1092  1288  1125]
Found 10 using 23 at locations [-2449  1277 -1238]
Found 3 using 32 at locations [-2290 -2266  -141]
Found 16 using 36 at locations [-4753    30  -166]
Found 28 using 36 at locations [-3532    97  1132]
Found 8 using 12 at locations [-1224  2573  1061]
F

In [27]:
solve_a(sample)


Found 1 at locations [   68 -1246   -43]


KeyboardInterrupt: 

In [18]:
solve_a(data)

Found 2 at locations [ -28 1245 -161]
Found 24 at locations [1163   26 -170]
Found 27 at locations [-1254    90  -158]
Found 18 at locations [  -18  1355 -1280]
Found 31 at locations [ 1146 -1193   -28]
Found 1 at locations [-2380    15   -13]
Found 4 at locations [-1078    63  1010]
Found 26 at locations [-1105    28 -1236]
Found 7 at locations [  -12  2400 -1318]
Found 23 at locations [-1075  1190 -1385]
Found 15 at locations [-2348   173 -1313]
Found 32 at locations [-2450 -1024   -26]
Found 36 at locations [-3547     0   -79]
Found 11 at locations [-1233 -1211  1144]
Found 12 at locations [-1092  1288  1125]
Found 10 at locations [-2449  1277 -1238]
Found 3 at locations [-2290 -2266  -141]
Found 16 at locations [-4753    30  -166]
Found 28 at locations [-3532    97  1132]
Found 8 at locations [-1224  2573  1061]
Found 30 at locations [-2428  1249  1069]
Found 6 at locations [-2269 -3556   -95]
Found 19 at locations [-1131 -2278  -172]
Found 20 at locations [-2337 -2411  1197]
Found

449

In [19]:
L = [
"[ -28 1245 -161]",
" [1163   26 -170]",
" [-1254    90  -158]",
" [  -18  1355 -1280]",
" [ 1146 -1193   -28]",
"[-2380    15   -13]",
"[-1078    63  1010]",
" [-1105    28 -1236]",
"[  -12  2400 -1318]",
" [-1075  1190 -1385]",
" [-2348   173 -1313]",
" [-2450 -1024   -26]",
" [-3547     0   -79]",
" [-1233 -1211  1144]",
" [-1092  1288  1125]",
" [-2449  1277 -1238]",
"[-2290 -2266  -141]",
" [-4753    30  -166]",
" [-3532    97  1132]",
"[-1224  2573  1061]",
" [-2428  1249  1069]",
"[-2269 -3556   -95]",
" [-1131 -2278  -172]",
" [-2337 -2411  1197]",
" [-4722    31  1198]",
" [-6042   159  -182]",
" [-3560   131  2245]",
"[-2385 -3457  1117]",
" [-2369 -4811   -22]",
" [-1144 -3615   -89]",
" [-4732   -17  2315]",
" [-5936 -1062   -10]",
" [-2339 -3488  2394]",
" [-2334 -4684  1017]",
"[-3472 -3589  2361]",
" [-1256 -3495  2220]",
]

In [24]:
def get(text):
    return np.array([[i.fixed[0] for i in parse.findall('{:d}', line)] for line in text])

In [31]:

locs = np.array([
    [   68, -1246,   -43],
    [  -92, -2380,   -20],
    [  -20, -1133,  1061],
    [ 1105, -1205,  1229],
])

In [32]:
def furthest(locs):
    return max(abs(i-j).sum() for i, j in itertools.combinations(locs, 2))
        

In [33]:
furthest(locs)

3621

In [34]:
locs = get(L)
furthest(locs)

13128

In [ ]:
o1-d = o2 + d

In [65]:
sample[0].shape

(25, 3)

In [67]:
sample[0].shape

(25, 3)

In [69]:
sample[2].shape

(26, 3)

In [72]:
(sample[0][:,None,:] - sample[2][None,:,:]).shape

(25, 26, 3)

In [73]:
np.unique(np.zeros((2,3,4)))

array([0.])

In [77]:
366 % 7

2

In [ ]:
@lru_cache()
def righthand_rotations():
    # Generate all 3! * 2**3 possible rotation/mirror matrices
    # and keep the ones that are rotation since det(R) = 1
    # Returns pairs of row permutation indices and signs, not matrices
    I = np.eye(3, dtype=int)
    row_perms = map(np.array, itertools.permutations(range(3)))
    signs = map(np.array, itertools.product((1,-1),repeat=3))
    return [(p,s) for p, s in itertools.product(row_perms, signs) if np.linalg.det(I[p]*s) == 1]

In [78]:
def valid_overlap(known, candidate):
    # known is absolute coordinates, candidate is relative/distances
    for (perm, sign) in righthand_rotations():
        rot_candidate = candidate[:,perm] * sign
        # known = candidate + loc_candidate, so by substractng we get a 2d array of vectors
        # containing the location of candidate if the i^th beacon matched the j^th distance
        pred_locations = known[:,None,:] - rot_candidate[None,:,:]
        loc, counts = np.unique(pred_locations.reshape(-1,3), axis=0, return_counts=True)
        candidate_loc = loc[counts >= 12]
        if len(candidate_loc) > 0:
            candidate_loc = candidate_loc[0]
            return candidate_loc, rot_candidate + candidate_loc

In [106]:
def find_absolute_positions(scanners):

    locations = {0: np.array([0,0,0])}
    absolute = {0: scanners[0]}
    unsolved = set(range(1,len(scanners)))
    tested = set()

    while len(unsolved) > 0:
        for known, candidate in itertools.product(absolute, unsolved):
            if (known, candidate) not in tested:
                tested.add((known, candidate))
                if (sol := valid_overlap(absolute[known], scanners[candidate])) is not None:
                    locations[candidate], absolute[candidate] = sol
                    unsolved.remove(candidate)
                    break
    return locations, absolute 

In [107]:
start = time.time()
find_absolute_positions(data)
print(time.time()-start)

3.8145127296447754


In [98]:
def find_absolute_positions(scanners):

    locations = [np.array([0,0,0])] + [None]*(len(scanners)-1)
    absolute = [scanners[0]] + [None]*(len(scanners)-1)
    all_absolute = scanners[0]
    unsolved = set(range(1,len(scanners)))

    while len(unsolved) > 0:
        for candidate in unsolved:
            if (sol := valid_overlap(all_absolute, scanners[candidate])) is not None:
                locations[candidate], absolute[candidate] = sol
                unsolved.remove(candidate)
                all_absolute = np.vstack([all_absolute, absolute[candidate]])
                break
                
    return locations, absolute 

In [99]:
start = time.time()
find_absolute_positions(data)
print(time.time()-start)

23.01850700378418


In [105]:
def solve_a(scanners):
    _, absolute = find_absolute_positions(scanners)
    return len(np.unique(np.vstack(absolute), axis=0))

In [104]:
def solve_b(scanners):
    locations, _ = find_absolute_positions(scanners)
    return max(abs(i-j).sum() for i, j in itertools.combinations(locations, 2))

In [ ]:
start = time.time()
solve(data)
print(time.time()-start)